In [ ]:
import sys
sys.path.append('../src')
import logging
logging.getLogger().setLevel(logging.INFO)
from sklearn.model_selection import train_test_split
import numpy as np
import os
import matplotlib.pyplot as plt
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch
from tqdm import tqdm
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.metrics.pairwise import rbf_kernel
from PIL import Image
import requests
import pickle
from transformers import CLIPProcessor, CLIPModel
import math
print(torch.cuda.is_available())
import pickle
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import matplotlib
from collections import Counter
import datetime
import torch
import time
# local imports
from teacher_methods.teacher_domino import *
from teacher_methods.teacher_kmeans import *
from teacher_methods.teacher_selection import *
from teacher_methods.teacher_gen import *
from datasets_hai.bdd import *
from datasets_hai.coco import *
from datasets_hai.mmlu import *
from datasets_hai.dynasent import *
from datasets_hai.gaussians import *
from utils.utils import *
from utils.metrics_hai import *
from utils.metrics_regions import *
from synth_regions import *


metrics_print = ['adjusted_rand_score', 'fowlkes_mallows_score']
def get_metrics_region_process(data_runs):
    results = {}
    for metric in metrics_print:
        list_metric = []
        for j in range(len(data_runs[0])):
            mean = np.mean([data_runs[i][j][metric] for i in range(len(data_runs))])
            std = np.std([data_runs[i][j][metric] for i in range(len(data_runs))])/np.sqrt(len(data_runs))
            list_metric.append((mean,std))
        results[metric] = list_metric
    return results


In [ ]:
alpha = 0.8
beta_high = 0.2
beta_low = 0.01
delta = 2
randomized_sampling = 1
parallel_processes = 1
kernel = rbf_kernel # K(.,.)
metric_y = loss_01
initialization_epochs = 200
initialization_restarts = 50
TOTAL_TRIALS = 5
DATA_SIZES = [8]
epochs = 2000
lr = 0.001


In [ ]:
metadata_dimensions = 1
GOOD_REGION_ACC = 0.95
BAD_REGION_ACC = 0.6
OTHER_REGION_ACC = 0.75
N_CLAUSES = random.randint(4, 4) # number of clauses in each region
CLAUSE_LENGTH = random.randint(1,1) # number of metadata dimensions in each clause

MIN_REGION_SIZE = 0.01 # minimum size of a region
MAX_REGION_SIZE = 0.2 # maximum size of a region




# BDD

In [ ]:
date_now = datetime.datetime.now()
date_now = date_now.strftime("%Y-%m-%d_%H%M%S")
dataset = pickle.load(open("../data/cleaned_pkl/bdd_dataset.pkl","rb"))
metadata_dimensions = len(dataset.metadata[0])


In [ ]:

# split data into 10% train and 90% test, get ids
data_size_n = len(dataset.data_y)



metrics_gen_region = []
metrics_domino_region = []
metrics_kmeans_region = []
metrics_selec_region = []

metrics_gen_test = []
metrics_domino_test = []
metrics_kmeans_test = []
metrics_selec_test = []

metrics_human_alone = []
metrics_ai_alone = []

max_teaching_points = max(DATA_SIZES)
region_names = []
for trial in range(TOTAL_TRIALS):
    data_ids = list(range(0,len(dataset.data_y) ))
    data_train_ids, data_test_ids = train_test_split(data_ids, test_size=0.3)

    
    ai_preds, hum_preds, true_regions, ai_regions_names, hum_regions_names = get_synth_ai_human_regions(dataset, metadata_dimensions, GOOD_REGION_ACC, BAD_REGION_ACC, OTHER_REGION_ACC, N_CLAUSES, CLAUSE_LENGTH, MIN_REGION_SIZE, MAX_REGION_SIZE)

    print(ai_regions_names)
    print(hum_regions_names)
    region_names.append([ai_regions_names,hum_regions_names])
    dataset.hum_preds = hum_preds
    dataset.ai_preds = ai_preds
    prior_rejector_preds = np.array([np.random.choice([0,1], p=[0.5,0.5]) for i in range(len(dataset.data_y))])


    teacher_gen = TeacherGenerative(dataset.data_x[data_train_ids], dataset.data_y[data_train_ids], dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids],
                                    prior_rejector_preds[data_train_ids], metric_y, max_teaching_points, alpha, beta_high, beta_low, delta)
    teacher_gen.epochs = epochs
    teacher_gen.lr = lr
    teacher_gen.initialization_restarts = initialization_restarts
    teacher_gen.initialization_epochs = initialization_epochs
    teacher_gen.fit()



    teacher_selection = TeacherSelective(dataset.data_x[data_train_ids], dataset.data_y[data_train_ids], dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], prior_rejector_preds[data_train_ids], kernel, metric_y, max_teaching_points, alpha, beta_high, beta_low,  randomized_sampling, delta, parallel_processes)
    print("SELECTION")
    teacher_selection.randomized_sampling = randomized_sampling
    teacher_selection.parallel_processes = parallel_processes
    teacher_selection.fit()

    saved_selec_points = copy.deepcopy(teacher_selection.teaching_set)
    saved_gen_points = copy.deepcopy(teacher_gen.teaching_set)
    # fit generative and selection
    metric_gen_region_trial = []
    metric_selec_region_trial = []
    metric_gen_test_trial = []
    metric_selec_test_trial = []
    metric_domino_region_trial = []
    metric_domino_test_trial = []
    metric_kmeans_region_trial = []
    metric_kmeans_test_trial = []
    metrics_human_alone_trial = []
    metrics_ai_alone_trial = []

    for data_size in DATA_SIZES:
        try:
            teacher_domino = TeacherDomino(dataset.data_x[data_train_ids], dataset.data_y[data_train_ids], dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], dataset.ai_scores[data_train_ids], metric_y, n_pca_components = None, n_mixture_components = data_size , teaching_points = data_size)
            teacher_domino.fit()
        except:
            print("Domino failed")
            teacher_domino = TeacherDomino(dataset.data_x[data_train_ids], dataset.data_y[data_train_ids], dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], dataset.ai_scores[data_train_ids], metric_y, n_pca_components = None, n_mixture_components = 50 , teaching_points = data_size)
            teacher_domino.fit()    
            
        teacher_kmeans = TeacherKmeans(dataset.data_x[data_train_ids], dataset.data_y[data_train_ids], dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], metric_y, data_size)
        print("Domino")
        teacher_domino.fit()
        teacher_kmeans.fit()
        teacher_selection.teaching_set = saved_selec_points[:data_size]
        teacher_gen.teaching_set = saved_gen_points[:data_size]
        # get eval results

        human_only = compute_metrics( dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], np.zeros(len(data_train_ids)), dataset.data_y[data_train_ids], metric_y)
        ai_only = compute_metrics( dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], np.ones(len(data_train_ids)), dataset.data_y[data_train_ids], metric_y)

        metrics_human_alone_trial.append(human_only)
        metrics_ai_alone_trial.append(ai_only)
        # TEST, same code but use data_test_ids
        domino_defer_preds = teacher_domino.get_defer_preds(dataset.data_x[data_test_ids])
        domino_loss = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], domino_defer_preds, dataset.data_y[data_test_ids], metric_y)
        kmeans_defer_preds = teacher_kmeans.get_defer_preds(dataset.data_x[data_test_ids])
        kmeans_loss = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], kmeans_defer_preds, dataset.data_y[data_test_ids], metric_y)
        selection_defer_preds = teacher_selection.get_defer_preds(dataset.data_x[data_test_ids], prior_rejector_preds[data_test_ids])
        selection_loss = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], selection_defer_preds, dataset.data_y[data_test_ids], metric_y)
        gen_defer_preds = teacher_gen.get_defer_preds(dataset.data_x[data_test_ids], prior_rejector_preds[data_test_ids])
        gen_loss = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], gen_defer_preds, dataset.data_y[data_test_ids], metric_y)
        human_only = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], np.zeros(len(data_test_ids)), dataset.data_y[data_test_ids], metric_y)
        ai_only = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], np.ones(len(data_test_ids)), dataset.data_y[data_test_ids], metric_y)
        metric_gen_test_trial.append(gen_loss)
        metric_selec_test_trial.append(selection_loss)
        metric_domino_test_trial.append(domino_loss)
        metric_kmeans_test_trial.append(kmeans_loss)
        # now get region preds on train
        gen_region_labels = teacher_gen.get_region_labels(dataset.data_x[data_train_ids], prior_rejector_preds[data_train_ids])
        selec_region_labels = teacher_selection.get_region_labels(dataset.data_x[data_train_ids], prior_rejector_preds[data_train_ids])
        domino_region_labels = teacher_domino.get_region_labels(dataset.data_x[data_train_ids])
        kmeans_region_labels = teacher_kmeans.get_region_labels(dataset.data_x[data_train_ids])
        gen_region_metrics = get_region_metrics(true_regions[data_train_ids], gen_region_labels)
        selec_region_metrics = get_region_metrics(true_regions[data_train_ids], selec_region_labels)
        domino_region_metrics = get_region_metrics(true_regions[data_train_ids], domino_region_labels)
        kmeans_region_metrics = get_region_metrics(true_regions[data_train_ids], kmeans_region_labels)
        metric_gen_region_trial.append(gen_region_metrics)
        metric_selec_region_trial.append(selec_region_metrics)
        metric_domino_region_trial.append(domino_region_metrics)
        metric_kmeans_region_trial.append(kmeans_region_metrics)

        
    metrics_human_alone.append(metrics_human_alone_trial)
    metrics_ai_alone.append(metrics_ai_alone_trial)
    metrics_gen_region.append(metric_gen_region_trial)
    metrics_selec_region.append(metric_selec_region_trial)
    metrics_domino_region.append(metric_domino_region_trial)
    metrics_kmeans_region.append(metric_kmeans_region_trial)
    metrics_gen_test.append(metric_gen_test_trial)
    metrics_selec_test.append(metric_selec_test_trial)
    metrics_domino_test.append(metric_domino_test_trial)
    metrics_kmeans_test.append(metric_kmeans_test_trial)
    data_save = {}
    data_save["metrics_human_alone"] = metrics_human_alone
    data_save["metrics_ai_alone"] = metrics_ai_alone
    data_save["metrics_gen_region"] = metrics_gen_region
    data_save["metrics_selec_region"] = metrics_selec_region
    data_save["metrics_domino_region"] = metrics_domino_region
    data_save["metrics_kmeans_region"] = metrics_kmeans_region
    data_save["metrics_gen_test"] = metrics_gen_test
    data_save["metrics_selec_test"] = metrics_selec_test
    data_save["metrics_domino_test"] = metrics_domino_test
    data_save["metrics_kmeans_test"] = metrics_kmeans_test
    data_save["DATA_SIZES"] = DATA_SIZES
    data_save["region_names"] = region_names
    data_save["TOTAL_TRIALS"] = TOTAL_TRIALS
    with open("../exp_data/results/bdd_synth_" + date_now + ".pkl", "wb") as f:
        pickle.dump(data_save, f)    
    print("trial", trial, "done")
         








In [ ]:

print("GEN")
print(get_metrics_region_process(metrics_gen_region))
print("SELEC")
print(get_metrics_region_process(metrics_selec_region))
print("DOMINO")
print(get_metrics_region_process(metrics_domino_region))
print("KMEANS")
print(get_metrics_region_process(metrics_kmeans_region))
alpha_overlap = 0.5

# printing from a pickle file
max_trials = 1# TOTAL_TRIALS
ns = DATA_SIZES


avgs_rand = [np.average([metrics_gen_test[triall][i][1]['score']   for triall in range(max_trials)]) for i in range(len(ns))]
stds_rand = [np.std([metrics_gen_test[triall][i][1]['score']     for triall in range(max_trials)]) for i in range(len(ns))]
plt.errorbar(ns,  avgs_rand, yerr=stds_rand, marker = "d",  label=f'GEN (ours)', alpha =alpha_overlap, color = "black")

avgs_rand = [np.average([metrics_domino_test[triall][i][1]['score']   for triall in range(max_trials)]) for i in range(len(ns))]
stds_rand = [np.std([metrics_domino_test[triall][i][1]['score']  for triall in range(max_trials)]) for i in range(len(ns))]
plt.errorbar(ns,  avgs_rand, yerr=stds_rand, marker = "o",  label=f'DOMINO', alpha =alpha_overlap, color = "red")

avgs_rand = [np.average([metrics_kmeans_test[triall][i][1]['score']   for triall in range(max_trials)]) for i in range(len(ns))]
stds_rand = [np.std([metrics_kmeans_test[triall][i][1]['score']  for triall in range(max_trials)]) for i in range(len(ns))]
plt.errorbar(ns,  avgs_rand, yerr=stds_rand, marker = "x",  label=f'KMEANS', alpha =alpha_overlap, color = "blue")


avgs_rand = [np.average([metrics_selec_test[triall][i][1]['score']     for triall in range(max_trials)]) for i in range(len(ns))]
stds_rand = [np.std([metrics_selec_test[triall][i][1]['score']     for triall in range(max_trials)]) for i in range(len(ns))]
plt.errorbar(ns,  avgs_rand, yerr=stds_rand, marker = "<",  label=f'SELECT', alpha =alpha_overlap, color = "green")





ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()   
plt.grid()
plt.legend(fontsize='xx-large')
plt.legend()
plt.ylabel('Test Loss ', fontsize='xx-large')
plt.xlabel('Training data size', fontsize='xx-large')
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 6
fig_size[1] = 4.2

#plt.savefig("../exp_data/plots/plot_synth_data_realizable_"+ date_now+".pdf", dpi = 1000, bbox_inches='tight')
#plt.show()
# 

# MMLU

TODO: fix synth_regions predictions to be random 

In [ ]:
date_now = datetime.datetime.now()
date_now = date_now.strftime("%Y-%m-%d_%H%M%S")
dataset =  pickle.load(open('../data/cleaned_pkl/mmlu_dataset.pkl',"rb"))
metadata_dimensions = len(dataset.metadata[0])

In [ ]:

# split data into 10% train and 90% test, get ids
data_size_n = len(dataset.data_y)


metrics_gen_region = []
metrics_domino_region = []
metrics_kmeans_region = []
metrics_selec_region = []

metrics_gen_test = []
metrics_domino_test = []
metrics_kmeans_test = []
metrics_selec_test = []

metrics_human_alone = []
metrics_ai_alone = []

max_teaching_points = max(DATA_SIZES)
region_names = []
for trial in range(TOTAL_TRIALS):
    data_ids = list(range(0,len(dataset.data_y) ))
    data_train_ids, data_test_ids = train_test_split(data_ids, test_size=0.3)

    
    ai_preds, hum_preds, true_regions, ai_regions_names, hum_regions_names = get_synth_ai_human_regions(dataset, metadata_dimensions, GOOD_REGION_ACC, BAD_REGION_ACC, OTHER_REGION_ACC, N_CLAUSES, CLAUSE_LENGTH, MIN_REGION_SIZE, MAX_REGION_SIZE)

    print(ai_regions_names)
    print(hum_regions_names)
    region_names.append([ai_regions_names,hum_regions_names])
    dataset.hum_preds = hum_preds
    dataset.ai_preds = ai_preds
    prior_rejector_preds = np.array([np.random.choice([0,1], p=[0.5,0.5]) for i in range(len(dataset.data_y))])


    teacher_gen = TeacherGenerative(dataset.data_x[data_train_ids], dataset.data_y[data_train_ids], dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids],
                                    prior_rejector_preds[data_train_ids], metric_y, max_teaching_points, alpha, beta_high, beta_low, delta)
    teacher_gen.epochs = epochs
    teacher_gen.lr = lr
    teacher_gen.initialization_restarts = initialization_restarts
    teacher_gen.initialization_epochs = initialization_epochs
    teacher_gen.fit()



    teacher_selection = TeacherSelective(dataset.data_x[data_train_ids], dataset.data_y[data_train_ids], dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], prior_rejector_preds[data_train_ids], kernel, metric_y, max_teaching_points, alpha, beta_high, beta_low,  randomized_sampling, delta, parallel_processes)
    print("SELECTION")
    teacher_selection.randomized_sampling = randomized_sampling
    teacher_selection.parallel_processes = parallel_processes
    teacher_selection.fit()

    saved_selec_points = copy.deepcopy(teacher_selection.teaching_set)
    saved_gen_points = copy.deepcopy(teacher_gen.teaching_set)
    # fit generative and selection
    metric_gen_region_trial = []
    metric_selec_region_trial = []
    metric_gen_test_trial = []
    metric_selec_test_trial = []
    metric_domino_region_trial = []
    metric_domino_test_trial = []
    metric_kmeans_region_trial = []
    metric_kmeans_test_trial = []
    metrics_human_alone_trial = []
    metrics_ai_alone_trial = []

    for data_size in DATA_SIZES:
        data_y_one_hot = np.zeros((len(data_train_ids), 4))
        dataset.data_y = dataset.data_y.astype(int)
        data_y_one_hot[np.arange(len(data_train_ids)), dataset.data_y[data_train_ids]] = 1
        ai_preds_one_hot = np.zeros((len(data_train_ids), 4))
        dataset.ai_preds = dataset.ai_preds.astype(int)
        ai_preds_one_hot[np.arange(len(data_train_ids)), dataset.ai_preds[data_train_ids]] = 1
        hum_preds_one_hot = np.zeros((len(data_train_ids), 4))
        dataset.hum_preds = dataset.hum_preds.astype(int)
        hum_preds_one_hot[np.arange(len(data_train_ids)), dataset.hum_preds[data_train_ids]] = 1
        try:
            teacher_domino = TeacherDomino(dataset.data_x[data_train_ids], data_y_one_hot,hum_preds_one_hot,ai_preds_one_hot, ai_preds_one_hot, metric_y, n_pca_components = None, n_mixture_components = data_size , teaching_points = data_size)
            teacher_domino.fit()
        except:
            print("Domino failed")
            teacher_domino = TeacherDomino(dataset.data_x[data_train_ids], data_y_one_hot,hum_preds_one_hot,ai_preds_one_hot, ai_preds_one_hot, metric_y, n_pca_components = None, n_mixture_components = 50 , teaching_points = data_size)
            teacher_domino.fit()     
            
        teacher_kmeans = TeacherKmeans(dataset.data_x[data_train_ids], dataset.data_y[data_train_ids], dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], metric_y, data_size)
        print("Domino")
        teacher_domino.fit()
        teacher_kmeans.fit()
        teacher_selection.teaching_set = saved_selec_points[:data_size]
        teacher_gen.teaching_set = saved_gen_points[:data_size]
        # get eval results

        human_only = compute_metrics( dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], np.zeros(len(data_train_ids)), dataset.data_y[data_train_ids], metric_y)
        ai_only = compute_metrics( dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], np.ones(len(data_train_ids)), dataset.data_y[data_train_ids], metric_y)

        metrics_human_alone_trial.append(human_only)
        metrics_ai_alone_trial.append(ai_only)
        # TEST, same code but use data_test_ids
        domino_defer_preds = teacher_domino.get_defer_preds(dataset.data_x[data_test_ids])
        domino_loss = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], domino_defer_preds, dataset.data_y[data_test_ids], metric_y)
        kmeans_defer_preds = teacher_kmeans.get_defer_preds(dataset.data_x[data_test_ids])
        kmeans_loss = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], kmeans_defer_preds, dataset.data_y[data_test_ids], metric_y)
        selection_defer_preds = teacher_selection.get_defer_preds(dataset.data_x[data_test_ids], prior_rejector_preds[data_test_ids])
        selection_loss = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], selection_defer_preds, dataset.data_y[data_test_ids], metric_y)
        gen_defer_preds = teacher_gen.get_defer_preds(dataset.data_x[data_test_ids], prior_rejector_preds[data_test_ids])
        gen_loss = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], gen_defer_preds, dataset.data_y[data_test_ids], metric_y)
        human_only = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], np.zeros(len(data_test_ids)), dataset.data_y[data_test_ids], metric_y)
        ai_only = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], np.ones(len(data_test_ids)), dataset.data_y[data_test_ids], metric_y)
        metric_gen_test_trial.append(gen_loss)
        metric_selec_test_trial.append(selection_loss)
        metric_domino_test_trial.append(domino_loss)
        metric_kmeans_test_trial.append(kmeans_loss)
        # now get region preds on train
        gen_region_labels = teacher_gen.get_region_labels(dataset.data_x[data_train_ids], prior_rejector_preds[data_train_ids])
        selec_region_labels = teacher_selection.get_region_labels(dataset.data_x[data_train_ids], prior_rejector_preds[data_train_ids])
        domino_region_labels = teacher_domino.get_region_labels(dataset.data_x[data_train_ids])
        kmeans_region_labels = teacher_kmeans.get_region_labels(dataset.data_x[data_train_ids])
        gen_region_metrics = get_region_metrics(true_regions[data_train_ids], gen_region_labels)
        selec_region_metrics = get_region_metrics(true_regions[data_train_ids], selec_region_labels)
        domino_region_metrics = get_region_metrics(true_regions[data_train_ids], domino_region_labels)
        kmeans_region_metrics = get_region_metrics(true_regions[data_train_ids], kmeans_region_labels)
        metric_gen_region_trial.append(gen_region_metrics)
        metric_selec_region_trial.append(selec_region_metrics)
        metric_domino_region_trial.append(domino_region_metrics)
        metric_kmeans_region_trial.append(kmeans_region_metrics)

        
    metrics_human_alone.append(metrics_human_alone_trial)
    metrics_ai_alone.append(metrics_ai_alone_trial)
    metrics_gen_region.append(metric_gen_region_trial)
    metrics_selec_region.append(metric_selec_region_trial)
    metrics_domino_region.append(metric_domino_region_trial)
    metrics_kmeans_region.append(metric_kmeans_region_trial)
    metrics_gen_test.append(metric_gen_test_trial)
    metrics_selec_test.append(metric_selec_test_trial)
    metrics_domino_test.append(metric_domino_test_trial)
    metrics_kmeans_test.append(metric_kmeans_test_trial)
    data_save = {}
    data_save["metrics_human_alone"] = metrics_human_alone
    data_save["metrics_ai_alone"] = metrics_ai_alone
    data_save["metrics_gen_region"] = metrics_gen_region
    data_save["metrics_selec_region"] = metrics_selec_region
    data_save["metrics_domino_region"] = metrics_domino_region
    data_save["metrics_kmeans_region"] = metrics_kmeans_region
    data_save["metrics_gen_test"] = metrics_gen_test
    data_save["metrics_selec_test"] = metrics_selec_test
    data_save["metrics_domino_test"] = metrics_domino_test
    data_save["metrics_kmeans_test"] = metrics_kmeans_test
    data_save["DATA_SIZES"] = DATA_SIZES
    data_save["region_names"] = region_names
    data_save["TOTAL_TRIALS"] = TOTAL_TRIALS
    with open("../exp_data/results/mmlu_synth_" + date_now + ".pkl", "wb") as f:
        pickle.dump(data_save, f)    
    print("trial", trial, "done")
         








In [ ]:

print("GEN")
print(get_metrics_region_process(metrics_gen_region))
print("SELEC")
print(get_metrics_region_process(metrics_selec_region))
print("DOMINO")
print(get_metrics_region_process(metrics_domino_region))
print("KMEANS")
print(get_metrics_region_process(metrics_kmeans_region))
alpha_overlap = 0.5

alpha_overlap = 0.5

# printing from a pickle file
max_trials = TOTAL_TRIALS
ns = DATA_SIZES


avgs_rand = [np.average([metrics_gen_test[triall][i][1]['score']   for triall in range(max_trials)]) for i in range(len(ns))]
stds_rand = [np.std([metrics_gen_test[triall][i][1]['score']     for triall in range(max_trials)]) for i in range(len(ns))]
plt.errorbar(ns,  avgs_rand, yerr=stds_rand, marker = "d",  label=f'GEN (ours)', alpha =alpha_overlap, color = "black")

avgs_rand = [np.average([metrics_domino_test[triall][i][1]['score']   for triall in range(max_trials)]) for i in range(len(ns))]
stds_rand = [np.std([metrics_domino_test[triall][i][1]['score']  for triall in range(max_trials)]) for i in range(len(ns))]
plt.errorbar(ns,  avgs_rand, yerr=stds_rand, marker = "o",  label=f'DOMINO', alpha =alpha_overlap, color = "red")

avgs_rand = [np.average([metrics_kmeans_test[triall][i][1]['score']   for triall in range(max_trials)]) for i in range(len(ns))]
stds_rand = [np.std([metrics_kmeans_test[triall][i][1]['score']  for triall in range(max_trials)]) for i in range(len(ns))]
plt.errorbar(ns,  avgs_rand, yerr=stds_rand, marker = "x",  label=f'KMEANS', alpha =alpha_overlap, color = "blue")


avgs_rand = [np.average([metrics_selec_test[triall][i][1]['score']     for triall in range(max_trials)]) for i in range(len(ns))]
stds_rand = [np.std([metrics_selec_test[triall][i][1]['score']     for triall in range(max_trials)]) for i in range(len(ns))]
plt.errorbar(ns,  avgs_rand, yerr=stds_rand, marker = "<",  label=f'SELECT', alpha =alpha_overlap, color = "green")





ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()   
plt.grid()
plt.legend(fontsize='xx-large')
plt.legend()
plt.ylabel('Test Loss ', fontsize='xx-large')
plt.xlabel('Training data size', fontsize='xx-large')
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 6
fig_size[1] = 4.2

#plt.savefig("../exp_data/plots/plot_synth_data_realizable_"+ date_now+".pdf", dpi = 1000, bbox_inches='tight')
#plt.show()
# 

# MS-COCO

In [ ]:
date_now = datetime.datetime.now()
date_now = date_now.strftime("%Y-%m-%d_%H%M%S")
dataset =  pickle.load(open('../data/cleaned_pkl/coco_dataset.pkl',"rb"))
metadata_dimensions = len(dataset.metadata[0])

In [ ]:

# split data into 10% train and 90% test, get ids
data_size_n = len(dataset.data_y)



metrics_gen_region = []
metrics_domino_region = []
metrics_kmeans_region = []
metrics_selec_region = []

metrics_gen_test = []
metrics_domino_test = []
metrics_kmeans_test = []
metrics_selec_test = []

metrics_human_alone = []
metrics_ai_alone = []

max_teaching_points = max(DATA_SIZES)
region_names = []
for trial in range(TOTAL_TRIALS):
    data_ids = list(range(0,len(dataset.data_y) ))
    data_train_ids, data_test_ids = train_test_split(data_ids, test_size=0.3)

    
    ai_preds, hum_preds, true_regions, ai_regions_names, hum_regions_names = get_synth_ai_human_regions(dataset, metadata_dimensions, GOOD_REGION_ACC, BAD_REGION_ACC, OTHER_REGION_ACC, N_CLAUSES, CLAUSE_LENGTH, MIN_REGION_SIZE, MAX_REGION_SIZE)

    print(ai_regions_names)
    print(hum_regions_names)
    region_names.append([ai_regions_names,hum_regions_names])
    dataset.hum_preds = hum_preds
    dataset.ai_preds = ai_preds
    prior_rejector_preds = np.array([np.random.choice([0,1], p=[0.5,0.5]) for i in range(len(dataset.data_y))])


    teacher_gen = TeacherGenerative(dataset.data_x[data_train_ids], dataset.data_y[data_train_ids], dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids],
                                    prior_rejector_preds[data_train_ids], metric_y, max_teaching_points, alpha, beta_high, beta_low, delta)
    teacher_gen.epochs = epochs
    teacher_gen.lr = lr
    teacher_gen.initialization_restarts = initialization_restarts
    teacher_gen.initialization_epochs = initialization_epochs
    teacher_gen.fit()



    teacher_selection = TeacherSelective(dataset.data_x[data_train_ids], dataset.data_y[data_train_ids], dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], prior_rejector_preds[data_train_ids], kernel, metric_y, max_teaching_points, alpha, beta_high, beta_low,  randomized_sampling, delta, parallel_processes)
    print("SELECTION")
    teacher_selection.randomized_sampling = randomized_sampling
    teacher_selection.parallel_processes = parallel_processes
    teacher_selection.fit()

    saved_selec_points = copy.deepcopy(teacher_selection.teaching_set)
    saved_gen_points = copy.deepcopy(teacher_gen.teaching_set)
    # fit generative and selection
    metric_gen_region_trial = []
    metric_selec_region_trial = []
    metric_gen_test_trial = []
    metric_selec_test_trial = []
    metric_domino_region_trial = []
    metric_domino_test_trial = []
    metric_kmeans_region_trial = []
    metric_kmeans_test_trial = []
    metrics_human_alone_trial = []
    metrics_ai_alone_trial = []

    for data_size in DATA_SIZES:
        try:
            teacher_domino = TeacherDomino(dataset.data_x[data_train_ids], dataset.data_y[data_train_ids], dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], dataset.ai_scores[data_train_ids], metric_y, n_pca_components = None, n_mixture_components = data_size , teaching_points = data_size)
            teacher_domino.fit()
        except:
            print("Domino failed")
            teacher_domino = TeacherDomino(dataset.data_x[data_train_ids], dataset.data_y[data_train_ids], dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], dataset.ai_scores[data_train_ids], metric_y, n_pca_components = None, n_mixture_components = 50 , teaching_points = data_size)
            teacher_domino.fit()    
            
        teacher_kmeans = TeacherKmeans(dataset.data_x[data_train_ids], dataset.data_y[data_train_ids], dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], metric_y, data_size)
        print("Domino")
        teacher_domino.fit()
        teacher_kmeans.fit()
        teacher_selection.teaching_set = saved_selec_points[:data_size]
        teacher_gen.teaching_set = saved_gen_points[:data_size]
        # get eval results

        human_only = compute_metrics( dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], np.zeros(len(data_train_ids)), dataset.data_y[data_train_ids], metric_y)
        ai_only = compute_metrics( dataset.hum_preds[data_train_ids], dataset.ai_preds[data_train_ids], np.ones(len(data_train_ids)), dataset.data_y[data_train_ids], metric_y)

        metrics_human_alone_trial.append(human_only)
        metrics_ai_alone_trial.append(ai_only)
        # TEST, same code but use data_test_ids
        domino_defer_preds = teacher_domino.get_defer_preds(dataset.data_x[data_test_ids])
        domino_loss = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], domino_defer_preds, dataset.data_y[data_test_ids], metric_y)
        kmeans_defer_preds = teacher_kmeans.get_defer_preds(dataset.data_x[data_test_ids])
        kmeans_loss = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], kmeans_defer_preds, dataset.data_y[data_test_ids], metric_y)
        selection_defer_preds = teacher_selection.get_defer_preds(dataset.data_x[data_test_ids], prior_rejector_preds[data_test_ids])
        selection_loss = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], selection_defer_preds, dataset.data_y[data_test_ids], metric_y)
        gen_defer_preds = teacher_gen.get_defer_preds(dataset.data_x[data_test_ids], prior_rejector_preds[data_test_ids])
        gen_loss = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], gen_defer_preds, dataset.data_y[data_test_ids], metric_y)
        human_only = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], np.zeros(len(data_test_ids)), dataset.data_y[data_test_ids], metric_y)
        ai_only = compute_metrics( dataset.hum_preds[data_test_ids], dataset.ai_preds[data_test_ids], np.ones(len(data_test_ids)), dataset.data_y[data_test_ids], metric_y)
        metric_gen_test_trial.append(gen_loss)
        metric_selec_test_trial.append(selection_loss)
        metric_domino_test_trial.append(domino_loss)
        metric_kmeans_test_trial.append(kmeans_loss)
        # now get region preds on train
        gen_region_labels = teacher_gen.get_region_labels(dataset.data_x[data_train_ids], prior_rejector_preds[data_train_ids])
        selec_region_labels = teacher_selection.get_region_labels(dataset.data_x[data_train_ids], prior_rejector_preds[data_train_ids])
        domino_region_labels = teacher_domino.get_region_labels(dataset.data_x[data_train_ids])
        kmeans_region_labels = teacher_kmeans.get_region_labels(dataset.data_x[data_train_ids])
        gen_region_metrics = get_region_metrics(true_regions[data_train_ids], gen_region_labels)
        selec_region_metrics = get_region_metrics(true_regions[data_train_ids], selec_region_labels)
        domino_region_metrics = get_region_metrics(true_regions[data_train_ids], domino_region_labels)
        kmeans_region_metrics = get_region_metrics(true_regions[data_train_ids], kmeans_region_labels)
        metric_gen_region_trial.append(gen_region_metrics)
        metric_selec_region_trial.append(selec_region_metrics)
        metric_domino_region_trial.append(domino_region_metrics)
        metric_kmeans_region_trial.append(kmeans_region_metrics)

        
    metrics_human_alone.append(metrics_human_alone_trial)
    metrics_ai_alone.append(metrics_ai_alone_trial)
    metrics_gen_region.append(metric_gen_region_trial)
    metrics_selec_region.append(metric_selec_region_trial)
    metrics_domino_region.append(metric_domino_region_trial)
    metrics_kmeans_region.append(metric_kmeans_region_trial)
    metrics_gen_test.append(metric_gen_test_trial)
    metrics_selec_test.append(metric_selec_test_trial)
    metrics_domino_test.append(metric_domino_test_trial)
    metrics_kmeans_test.append(metric_kmeans_test_trial)
    data_save = {}
    data_save["metrics_human_alone"] = metrics_human_alone
    data_save["metrics_ai_alone"] = metrics_ai_alone
    data_save["metrics_gen_region"] = metrics_gen_region
    data_save["metrics_selec_region"] = metrics_selec_region
    data_save["metrics_domino_region"] = metrics_domino_region
    data_save["metrics_kmeans_region"] = metrics_kmeans_region
    data_save["metrics_gen_test"] = metrics_gen_test
    data_save["metrics_selec_test"] = metrics_selec_test
    data_save["metrics_domino_test"] = metrics_domino_test
    data_save["metrics_kmeans_test"] = metrics_kmeans_test
    data_save["DATA_SIZES"] = DATA_SIZES
    data_save["region_names"] = region_names
    data_save["TOTAL_TRIALS"] = TOTAL_TRIALS
    with open("../exp_data/results/coco_synth_" + date_now + ".pkl", "wb") as f:
        pickle.dump(data_save, f)    
    print("trial", trial, "done")
         








In [ ]:
print("GEN")
print(get_metrics_region_process(metrics_gen_region))
print("SELEC")
print(get_metrics_region_process(metrics_selec_region))
print("DOMINO")
print(get_metrics_region_process(metrics_domino_region))
print("KMEANS")
print(get_metrics_region_process(metrics_kmeans_region))
alpha_overlap = 0.5

# printing from a pickle file
max_trials = TOTAL_TRIALS
ns = DATA_SIZES


avgs_rand = [np.average([metrics_gen_test[triall][i][1]['score']   for triall in range(max_trials)]) for i in range(len(ns))]
stds_rand = [np.std([metrics_gen_test[triall][i][1]['score']     for triall in range(max_trials)]) for i in range(len(ns))]
plt.errorbar(ns,  avgs_rand, yerr=stds_rand, marker = "d",  label=f'GEN (ours)', alpha =alpha_overlap, color = "black")

avgs_rand = [np.average([metrics_domino_test[triall][i][1]['score']   for triall in range(max_trials)]) for i in range(len(ns))]
stds_rand = [np.std([metrics_domino_test[triall][i][1]['score']  for triall in range(max_trials)]) for i in range(len(ns))]
plt.errorbar(ns,  avgs_rand, yerr=stds_rand, marker = "o",  label=f'DOMINO', alpha =alpha_overlap, color = "red")

avgs_rand = [np.average([metrics_kmeans_test[triall][i][1]['score']   for triall in range(max_trials)]) for i in range(len(ns))]
stds_rand = [np.std([metrics_kmeans_test[triall][i][1]['score']  for triall in range(max_trials)]) for i in range(len(ns))]
plt.errorbar(ns,  avgs_rand, yerr=stds_rand, marker = "x",  label=f'KMEANS', alpha =alpha_overlap, color = "blue")


avgs_rand = [np.average([metrics_selec_test[triall][i][1]['score']     for triall in range(max_trials)]) for i in range(len(ns))]
stds_rand = [np.std([metrics_selec_test[triall][i][1]['score']     for triall in range(max_trials)]) for i in range(len(ns))]
plt.errorbar(ns,  avgs_rand, yerr=stds_rand, marker = "<",  label=f'SELECT', alpha =alpha_overlap, color = "green")





ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()   
plt.grid()
plt.legend(fontsize='xx-large')
plt.legend()
plt.ylabel('Test Loss ', fontsize='xx-large')
plt.xlabel('Training data size', fontsize='xx-large')
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 6
fig_size[1] = 4.2

#plt.savefig("../exp_data/plots/plot_synth_data_realizable_"+ date_now+".pdf", dpi = 1000, bbox_inches='tight')
#plt.show()
# 